In [5]:
import json
from functools import lru_cache
from operator import itemgetter
from typing import List
import boto3
import pandas as pd
import requests
from datetime import datetime
from pydantic import BaseModel, confloat, AnyUrl,conint
from ratelimit import RateLimitException, limits
from tenacity import *
from yarl import URL

In [106]:
datetime.now().timestamp()

1648553556.521183

In [7]:
API_ENDPOINT = URL("https://api.coingecko.com/api/v3")

In [8]:
session = boto3.Session(profile_name='master')

In [9]:
@retry(
    retry=retry_if_exception_type(requests.exceptions.RequestException),
    wait=wait_exponential(max=60),
    stop=stop_after_attempt(3),
)
@retry(
    retry=retry_if_exception_type(RateLimitException),
    wait=wait_fixed(60),
    stop=stop_after_delay(360),
)
@limits(calls=50, period=60)
def call_get(url: URL, params: dict) -> str:
    return requests.get(url.update_query(params)).text

In [5]:
class MarketDataInfo(BaseModel):
    id: str
    symbol: str
    current_price: float
    market_cap: int

In [149]:
params = {
    'vs_currency' : "usd",
    'days' : "1"
}
res = json.loads(call_get(API_ENDPOINT / "coins" / 'bitcoin' / 'ohlc', params))

In [150]:
df = pd.DataFrame(res, columns=['timestamp', 'open', 'high', 'low', 'close'])
df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
df['coin'] = 'bitcoin'
df['formated_date'] = df['datetime'].dt.strftime('%Y-%m-%d')
df

,timestamp,open,high,low,close,datetime,coin,formated_date
0,1648472400000,47124.07,47137.14,47124.07,47137.14,2022-03-28 13:00:00,bitcoin,2022-03-28
1,1648474200000,47138.69,47218.09,47138.69,47218.09,2022-03-28 13:30:00,bitcoin,2022-03-28
2,1648476000000,47251.75,47600.85,47251.75,47600.85,2022-03-28 14:00:00,bitcoin,2022-03-28
3,1648477800000,47626.44,47626.44,47285.95,47301.04,2022-03-28 14:30:00,bitcoin,2022-03-28
4,1648479600000,47355.00,47557.11,47355.00,47557.11,2022-03-28 15:00:00,bitcoin,2022-03-28
5,1648481400000,47470.57,47664.42,47470.57,47655.46,2022-03-28 15:30:00,bitcoin,2022-03-28
6,1648483200000,47678.01,47678.01,47430.84,47430.84,2022-03-28 16:00:00,bitcoin,2022-03-28
7,1648485000000,47423.06,47452.47,47322.60,47423.43,2022-03-28 16:30:00,bitcoin,2022-03-28
8,1648486800000,47403.92,47450.37,47403.92,47450.37,2022-03-28 17:00:00,bitcoin,2022-03-28
9,1648488600000,47389.60,47389.60,47304.76,47359.06,2022-03-28 17:30:00,bitcoin,2022-03-28


In [136]:
r2 = json.loads(call_get(API_ENDPOINT / "coins" / 'bitcoin' / "market_chart", {"vs_currency" : "usd", "days" : "max"}))

In [ ]:
def currencies_info(data: dict, list_of_keys: List[str]):
    filtered_json = {key: data[key] for key in list_of_keys}
    temp_df = pd.json_normalize(filtered_json)
    temp_df['timestamp'] = datetime.now().timestamp()


In [40]:
res.keys()

dict_keys(['id', 'symbol', 'name', 'asset_platform_id', 'platforms', 'block_time_in_minutes', 'hashing_algorithm', 'categories', 'public_notice', 'additional_notices', 'description', 'links', 'image', 'country_origin', 'genesis_date', 'sentiment_votes_up_percentage', 'sentiment_votes_down_percentage', 'market_cap_rank', 'coingecko_rank', 'coingecko_score', 'developer_score', 'community_score', 'liquidity_score', 'public_interest_score', 'market_data', 'developer_data', 'public_interest_stats', 'status_updates', 'last_updated'])

In [93]:
list_of_keys = ['id', 'symbol', 'name', 'hashing_algorithm', 'categories', 'genesis_date',  'sentiment_votes_up_percentage',
                'sentiment_votes_down_percentage', 'market_cap_rank', 'coingecko_rank', 'coingecko_score', 'developer_score',
                'community_score', 'liquidity_score', 'public_interest_score', 'last_updated']

In [94]:
output = {key: res[key] for key in list_of_keys}

In [96]:
pd.json_normalize(output)

,id,symbol,name,hashing_algorithm,categories,genesis_date,sentiment_votes_up_percentage,sentiment_votes_down_percentage,market_cap_rank,coingecko_rank,coingecko_score,developer_score,community_score,liquidity_score,public_interest_score,last_updated
0,bitcoin,btc,Bitcoin,SHA-256,[Cryptocurrency],2009-01-03,89.43,10.57,1,1,81.178,98.923,74.645,100.056,0.28,2022-03-29T10:47:44.334Z


In [10]:
# list_of_coins = pd.read_csv("/home/vlad/master_project/data/coin_list/list_of_coins.csv")['id'].to_list()

In [9]:
def get_json_from_s3(bucket, key, session):
    s3 = session.client('s3')
    obj = s3.get_object(Bucket=bucket, Key=key)
    data = obj['Body'].read().decode('utf-8')
    return json.loads(data)

In [ ]:
session = boto3.Session(profile_name='master')